In [2]:
import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
import tensorflow.keras.backend as K
from commai_spd_chl_keras_datagen import commai_speed_chl_gen
from tensorflow.keras.layers import Dense,Activation,Lambda,GRU,BatchNormalization,Activation,Flatten,Conv2D,MaxPooling2D,Reshape,Dropout

Using TensorFlow backend.


In [2]:
tf.config.list_physical_devices ()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.keras.backend.image_data_format()

'channels_last'

In [4]:
seq = Sequential()

#should I trust this guy?https://www.kdnuggets.com/2018/09/dropout-convolutional-networks.html

seq.add(Lambda(lambda x:tf.divide(x,255),input_shape=(480*2,640,1))) #scale the inputs between 0-1

seq.add(Conv2D(filters=32,kernel_initializer='he_normal', kernel_size=(4,4),padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=32, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Flatten())

seq.add(Dense(192,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())
seq.add(Dropout(.3)) # throw away 30%

seq.add(Dense(50,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(Dropout(.2))

seq.add(Dense(2,kernel_initializer='he_normal'))

seq.add(Reshape((2,1))) #this sucks dick(s)

seq.compile(loss='mse', optimizer='adam')
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 960, 640, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 960, 640, 32)      544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 240, 160, 32)      0         
_________________________________________________________________
activation (Activation)      (None, 240, 160, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 240, 160, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 240, 160, 32)      16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 40, 32)        0

In [4]:
batch_size=32
n_e=60
n_frames_train=15000
n_frames_test=2900
n_frames_valid=2500

train_idx,test_idx,valid_idx=commai_speed_chl_gen(n_frames_valid=n_frames_valid,frames_per_sample=2,mode="stacked",
                    n_frames_test=n_frames_test,n_frames_train=n_frames_train,batch_size=batch_size).get_indexes()

load=np.load(open("idxes_21bs_STACKED_15k.np","rb"))
train_idx=load["train_idx"]
test_idx=load["test_idx"]
valid_idx=load["valid_idx"]

train_gen=commai_speed_chl_gen(frame_idx=train_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)
test_gen=commai_speed_chl_gen(frame_idx=test_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)
valid_gen=commai_speed_chl_gen(frame_idx=valid_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)

#np.savez_compressed(open("idxes_21bs_STACKED_15k.np","wb+"),train_idx=train_idx,test_idx=test_idx,valid_idx=valid_idx)



(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
train_samples: 7500 
test_samples:1450 
valid_samples: 1250
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)


In [ ]:
ie=0
filepath="STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-{epoch:02d}-{val_loss:.2f}-{loss:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

tb=tf.keras.callbacks.TensorBoard(
    log_dir='STACKED_CNN_2FPS_Block_approach_32bs-HE_norms', histogram_freq=1, write_graph=True, write_images=False,
    update_freq=10, profile_batch=0, embeddings_freq=0
)

cbs=[checkpoint,tb]
hst=seq.fit_generator(train_gen, epochs=n_e,workers=30,max_queue_size=100,
        use_multiprocessing=True,validation_data=test_gen,callbacks=cbs,verbose=1,shuffle=True, initial_epoch=ie)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
234/234 [==============================] - ETA: 0s - loss: 89.1752WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-01-117.33-89.18.hdf5
234/234 [==============================] - 148s 631ms/step - loss: 89.1752 - val_loss: 117.3281
Epoch 2/60
234/234 [==============================] - ETA: 0s - loss: 52.5207WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-02-55.56-52.52.hdf5
234/234 [==============================] - 152s 651ms/step - loss: 52.5207 - val_loss: 55.5624
Epoch 3/60
234/234 [==============================] - 

Epoch 9/60
234/234 [==============================] - ETA: 0s - loss: 13.5338WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00009: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-09-8.62-13.53.hdf5
234/234 [==============================] - 155s 664ms/step - loss: 13.5338 - val_loss: 8.6166
Epoch 10/60
234/234 [==============================] - ETA: 0s - loss: 13.7667WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00010: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-10-10.99-13.77.hdf5
234/234 [==============================] - 148s 634ms/step - loss: 13.7667 - val_loss: 10.9934
Epoch 11/60
234/234 [==============================] - ETA: 0s - loss: 12.6594WARNING:tensorflow:multiprocessing can interact badly w

234/234 [==============================] - ETA: 0s - loss: 8.3569WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00017: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-17-12.09-8.36.hdf5
234/234 [==============================] - 154s 659ms/step - loss: 8.3569 - val_loss: 12.0945
Epoch 18/60
234/234 [==============================] - ETA: 0s - loss: 8.4033WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00018: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-18-9.25-8.40.hdf5
234/234 [==============================] - 152s 651ms/step - loss: 8.4033 - val_loss: 9.2489
Epoch 19/60
234/234 [==============================] - ETA: 0s - loss: 8.3669WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, ca

234/234 [==============================] - ETA: 0s - loss: 7.1138WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00025: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-25-27.78-7.11.hdf5
234/234 [==============================] - 151s 643ms/step - loss: 7.1138 - val_loss: 27.7794
Epoch 26/60
234/234 [==============================] - ETA: 0s - loss: 7.4810WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00026: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-26-8.12-7.48.hdf5
234/234 [==============================] - 155s 663ms/step - loss: 7.4810 - val_loss: 8.1204
Epoch 27/60
234/234 [==============================] - ETA: 0s - loss: 7.0091WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, ca


Epoch 00033: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-33-8.76-6.48.hdf5
234/234 [==============================] - 156s 667ms/step - loss: 6.4755 - val_loss: 8.7625
Epoch 34/60
234/234 [==============================] - ETA: 0s - loss: 6.0176WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00034: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-34-10.86-6.02.hdf5
234/234 [==============================] - 147s 629ms/step - loss: 6.0176 - val_loss: 10.8632
Epoch 35/60
234/234 [==============================] - ETA: 0s - loss: 5.7582WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00035: saving model to STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-35-14.53-5.76.hdf5
234/234 [==============================] - 1

Process Keras_worker_ForkPoolWorker-4214:
Process Keras_worker_ForkPoolWorker-4222:
Process Keras_worker_ForkPoolWorker-4203:
Process Keras_worker_ForkPoolWorker-4226:
Process Keras_worker_ForkPoolWorker-4123:
Process Keras_worker_ForkPoolWorker-4220:
Process Keras_worker_ForkPoolWorker-4121:
Process Keras_worker_ForkPoolWorker-4210:
Process Keras_worker_ForkPoolWorker-4212:
Process Keras_worker_ForkPoolWorker-4219:
Process Keras_worker_ForkPoolWorker-4128:
Process Keras_worker_ForkPoolWorker-4125:
Process Keras_worker_ForkPoolWorker-4218:
Process Keras_worker_ForkPoolWorker-4134:
Process Keras_worker_ForkPoolWorker-4205:
Process Keras_worker_ForkPoolWorker-4140:
Process Keras_worker_ForkPoolWorker-4221:
Process Keras_worker_ForkPoolWorker-4116:
Process Keras_worker_ForkPoolWorker-4113:
Process Keras_worker_ForkPoolWorker-4138:
Process Keras_worker_ForkPoolWorker-4127:
Process Keras_worker_ForkPoolWorker-4118:
Process Keras_worker_ForkPoolWorker-4131:
Process Keras_worker_ForkPoolWorke

  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  Fi

  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/process.p

  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessin

  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 398, in _send_bytes
    self._send(buf)
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.p

In [5]:
#30 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_32bs-HE_norm-30-7.01-5.60.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

Instructions for updating:
Please use Model.evaluate, which supports generators.
40/39 - 10s - loss: 6.8383


6.838307857513428